# Pseudo-Synchronization

This adds self consistent spin, tidal and dynamical equations of motion in the equilibrium tide model as outlined by Eggleton et. al (1998). In this simple example we confirm the prediction of pseudo-synchronization for a fiducial hot Jupiter. This example can be seen in Millholland & Laughlin (2019) and is Figure 2 in Lu et al. (2023).

In [1]:
import rebound
import reboundx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

We'll set up the system in the traditional REBOUND framework first. We set up a fiducial hot Jupiter in a near-circular orbit around a solar-type star

In [7]:
sim = rebound.Simulation()
# Star
ms = 1
rs = 0.00465
sim.add(m = ms, r = rs)

# Planet
mp = 9.55e-4
rp = 4.676e-4
ap = 0.04072
ep = 0.01
sim.add(m = mp, r = rp, a = ap, e = ep)
ps = sim.particles

In [4]:
1 / (ps[1].n * 1e6)

8.213048308806583e-09

Now, we add the extra parameters necessary to calculate the spin and tidal effects. Let's first set the Love number, moment of inertia, and spin vector. In this simulation both bodies have structure.

In [3]:
rebx = reboundx.Extras(sim)
sf = rebx.load_force("spin")
rebx.add_force(sf)

# Star parameters
ps[0].params['k2'] = 0.07
ps[0].params['moi'] = 0.07 * ms * rs**2
# The magnitude of the spin axis is the spin frequency. We initialize the star's obliquity as 0
spin_period_star = (27 / 365) * 2 * np.pi # 27 days in default REBOUND units
spin_mag_star = 2 * np.pi / spin_period_star
ps[0].params['spin_sx'] = 0 
ps[0].params['spin_sy'] = 0
ps[0].params['spin_sz'] = spin_mag_star

# Planet parameters
ps[1].params['k2'] = 0.3
ps[1].params['moi'] = 0.25 * mp * rp**2
# Initialize the planet's obliquity to 30 degrees
spin_period_planet = (0.5 / 365) * 2 * np.pi # 0.5 days in default REBOUND units
spin_mag_planet = 2 * np.pi / spin_period_planet
obliquity_planet = np.radians(30)
ps[1].params['spin_sx'] = spin_mag_planet * np.sin(obliquity_planet)
ps[1].params['spin_sy'] = spin_mag_planet * np.sin(obliquity_planet)
ps[1].params['spin_sz'] = spin_mag_planet * np.cos(obliquity_planet)

We saved the tidal dissipation parameter $\sigma$ for last because there are a few different ways to set it. $\sigma$ may be set traditionally, but this parameter is not widely adopted. We've therefore offered two helper functions that relate it to two more traditionally used quantities: the time lag $\tau$ and the tidal quality factor $Q$. See Section 2 of Lu et. al (2023) for when these approximations are valid to use.

While setting $\sigma$ traditionally has no restrictions, to use the helper functions all other relevant quantities must be set already. Therefore, call these LAST.

UNFORTUNATELY HELPER FUNCTIONS NOT WORKING IN PYTHON YET...

In [4]:
# Setting sigma for the star: all three of these functions do the same thing
ps[0].params['sigma'] = 3.598e4
# rebx.set_q(sim.G, ps[0], ps[1], 1e6)
# rebx.set_time_lag(sim.G, ps[0], 1889)

ps[1].params['sigma'] = 8.164e10
# rebx.set_q(sim.G, ps[1], ps[0], 1e4)
# rebx.set_time_lag(sim.G, ps[1], 3.422e-7)

Some bookkeeping before we begin integrating the system: choosing an integrator, moving to the CoM, and aligning the system to the invariant plane.

In [ ]:
sim.integrator = "whfast"
sim.dt = 1e-3
sim.move_to_com()
rebx.align_simulation()

The next step is mandatory: to track the evolution of the spin axis, you MUST call the function `initialize_spin_ode`. Otherwise the spin axis will remain stationary and you won't see anything interesting!

DOESN'T WORK YET. CAN"T MANUALLY SET EITHER B/C CAN'T SET PRE/POST TIMESTEP

In [6]:
rebx.initialize_spin_ode(sim, sf)

Now, we integrate. We'll track the hot Jupiters's spin axis, semimajor axis, and eccentricity.

In [7]:
tmax = 2 * np.pi * 1e5
Nout = 100000
times = np.linspace(0, tmax, Nout)

hj_spin_axis = np.zeros((Nout, 3))
hj_orbit = np.zeros((Nout, 2))

for i, t in tqdm(enumerate(times)):
    hj_spin_axis[i] = [ps[1].params['spin_sx'], ps[1].params['spin_sy'], ps[1].params['spin_sz']]
    hj_orbit = [ps[1].a, ps[1].e]
    
    sim.integrate(t)

0it [00:00, ?it/s]/Users/tigerlu/Documents/School/mardling/rebound/rebound/simulation.py:724: RuntimeWarning: Spin axes are not being evolved. Call rebx_spin_initialize_ode to evolve

  warnings.warn(msg[1:], RuntimeWarning)
100000it [06:40, 249.66it/s]


Let's compare this to analytic results. We'll numerically integrate the same initial conditions with the orbit-averaged expressions from Leconte et. al (2010), which is an equivalent framework to ours.

In [10]:
from scipy.integrate import odeint

def leconte_analytic(params, t, mstar, rstar, k2_star, dt_star, C_s, mp, rp, k2_p, dt_p, C_p):
    '''
    Leconte 2010 Eq (2, 6, 10)
    Assumes REBOUND units and G=1
    '''
    
    a, e, omega_p, omega_s, ep_p, ep_s = params
    n = np.sqrt((mstar + mp) / a**3)
    
    n_e = (1 + (15 / 2) * e**2 + (45 / 8) * e**4 + (5 / 16) * e**6) / (1 - e**2)**6
    n_a_e = (1 + (31 / 2) * e**2 + (255 / 8) * e**4 + (185 / 16) * e**6 + (25 / 64) * e**8) / (1 -  e**2)**(15 / 2)
    n_e_e = (1 + (15 / 4) * e**2 + (15 / 8) * e**4 + (5 / 64) * e**6) / (1 - e**2)**(13 / 2)
    omega_e_e = (1 + (3 / 2) * e**2 + (1 / 8) * e**4) / (1 - e**2)**5
    omega_e = (1 + 3 * e**2 + (3 / 8) * e**4) / (1 - e**2)**(9 / 2)
    
    Kp = 1.5 * k2_p * dt_p * (mp**2 / rp) * (mstar / mp)**2 * (rp / a)**6 * n**2
    Kstar = 1.5 * k2_star * dt_star * (mstar**2 / rstar) * (mp / mstar)**2 * (rstar / a)**6 * n**2
    
    eta_p = (mp + mstar) / (mp * mstar) * (C_p * omega_p) / (a**2 * n * np.sqrt(1 - e**2))
    eta_s = (mp + mstar) / (mp * mstar) * (C_s * omega_s) / (a**2 * n * np.sqrt(1 - e**2))
    
    da_dt = (4 * a**2 / (mstar * mp)) * (Kp * (n_e * np.cos(ep_p) * omega_p / n - n_a_e)
                                       + Kstar * (n_e * np.cos(ep_s) * omega_s / n - n_a_e))
    
    de_dt = (11 * a * e / (mstar * mp)) * (Kp * (omega_e_e * np.cos(ep_p) * omega_p / n - (18 / 11) * n_e_e)
                                         + Kstar * (omega_e_e * np.cos(ep_s) * omega_s / n - (18 / 11) * n_e_e))
    
    domega_p_dt = -(Kp / (n * C_p)) * ((1 + np.cos(ep_p)**2) * omega_e * omega_p / n - 2 * np.cos(ep_p) * n_e)
    domega_s_dt = -(Kstar / (n * C_s)) * ((1 + np.cos(ep_s)**2) * omega_e * omega_s / n - 2 * np.cos(ep_s) * n_e)
    
    dep_p_dt = np.sin(ep_p) * ((Kp / (n * omega_p * C_p)) * 
                               ((np.cos(ep_p) - eta_p) * omega_e * omega_p / n - 2 * n_e))
    dep_s_dt = np.sin(ep_s) * ((Kstar / (n * omega_s * C_s)) * 
                               ((np.cos(ep_s) - eta_s) * omega_e * omega_s / n - 2 * n_e))
    
    return [da_dt, de_dt, domega_p_dt, domega_s_dt, dep_p_dt, dep_s_dt]

initial_condition = [0.04072, 0.01, 365 / 0.5, 365 / 27, np.radians(29.99488), np.radians(0)]
mm = np.sqrt((1 + 9.55e-4) / (0.04072**3))
print(mm)
print(1 / (mm * 10000.))
t = np.linspace(0, 2 * np.pi * 100000, 100000)
analytic_solution = odeint(lec_der, initial_condition, t,
                              args=(1., 0.00465, 0.07, 1 / (mm * 1000000.), 0.07 * 0.00465**2,
                                    9.55e-4, 4.676e-4, 0.3, 1 / (mm * 10000.), 0.25 * 9.55e-4 * (4.676e-4)**2))

121.75747206158916
8.213048308806587e-07


NameError: name 'lec_der' is not defined

In [12]:
(81643074533.4855 * 3 * rp**5 * 0.3) / (4 * sim.G)

4.106524154403292e-07

In [16]:
1 / (ps[1].n * 1e4)

8.213048308806582e-07